In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import scipy.stats as stats
import nltk.metrics as nm
import editdistance
import re
import matplotlib.pyplot as plt
execfile("utils.py")
%matplotlib inline

Couldn't import dot_parser, loading of dot files will not be possible.


In [60]:
# Load Network

G = nx.read_graphml('../save/nber.graphml')

In [86]:
# Filter on Date Range

init = 0
end = 2016

nodes = G.nodes()
edges = G.edges()

def filterDateRange(G, init, end, datevar='year'):
    return nx.MultiGraph([(u,v,d) for u,v,d in G.edges_iter(data=True) \
                          if d[datevar] >= init and d[datevar] <= end])

# Compute number of papers published total (i.e. # of edges) for each author

def nPapers(G, author):
    return len(set([d['title'] for u,v,d in G.edges_iter(data=True) \
                    if u == author or v == author]))

In [116]:
df = pd.DataFrame([(u,v,d['title']) for u,v,d in G.edges_iter(data=True)], \
                 columns = ['authorx','authory','title'])
left = df.groupby('authorx').size().reset_index()
right = df.groupby('authory').size().reset_index()
authdf = pd.merge(left, right, how='outer', left_on='authorx', right_on='authory')
authdf['author'] = authdf.authorx
authdf.loc[pd.isnull(authdf.author),'author'] = authdf['authory'][pd.isnull(authdf.author)]
authdf['nPapers'] = (authdf['0_x'].fillna(0) + authdf['0_y'].fillna(0)) / 2.
authdf

,authorx,0_x,authory,0_y,author,nPapers
0,ABovenberg,8,ABovenberg,10,ABovenberg,9
1,ABurnside,4,ABurnside,4,ABurnside,4
2,ACameron,2,ACameron,8,ACameron,5
3,ACoar,2,ACoar,4,ACoar,3
4,AFendrick,8,NaN,NaN,AFendrick,4
5,AGallant,6,AGallant,4,AGallant,5
6,AGner,2,AGner,2,AGner,2
7,ALBovenberg,2,ALBovenberg,2,ALBovenberg,2
8,AMacKinlay,6,AMacKinlay,6,AMacKinlay,6
9,AMobarak,6,NaN,NaN,AMobarak,3


In [4]:
#def filterEdge(edge, init, end):
#    return nx[edge[0]][edge[1]]

e = G[edges[0][0]][edges[0][1]]
[e[i]['year'] > 2010 for i in e.keys()]

[True, True]

In [5]:
edge = edges[0]
G[edge[0]][edge[1]]

{0: {'jel': u'D0,D8,G1,G3', 'year': 2012},
 1: {'jel': u'D0,D8,G1,G3', 'year': 2012}}

In [14]:
red_fish = set(n for u,v,d in G.edges_iter(data=True)
               if d['year'] > 2005
               for n in (u, v))

In [20]:
nx.MultiGraph([(u,v,d) for u,v,d in G.edges_iter(data=True) if d['year'] > 2005])

In [49]:
x = pd.read_csv('../save/NBER_Paper_Info.csv', delimiter='|')

In [52]:
x[['year','jel']].shape

(17728, 2)

In [57]:
x[['authors','year','jel']].drop_duplicates().shape

(17370, 3)

In [58]:
x[['authors','year','jel']].shape

(17728, 3)

In [59]:
x.groupby('authors')